In [1]:
# Take a manually created catalog and convert this to a segmantation map 
# by growing regions of pixels (in 3D) until their flux drops below
# N x noise(wl)

In [2]:
# go wide screen
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
from astropy.io import fits, ascii
from matplotlib import pyplot as plt
% matplotlib inline
from astropy.table import Table
import numpy as np
from astropy.wcs import WCS
import argparse

import cloud_finder3 as cf

In [9]:
def apply_manual_catalog(c,m,d,w,p, sigma_grow_threshold  = 1.7, maxlabel = 10000):
    print("Maxlabel is: ", maxlabel)

    count_labeled = 0
    s = "{:3s} {:3s} {:3s} {:3s} {:8s} {:7s} {:5s} {:6s}"
    print( s.format( "cidx", "x", "y", "z", "val", "wl[A]", "val/noise", "N_labeled"))
    s = "==================================="
    print( s )

    new_regs = []

    for i,r in enumerate(c):
        x,y,z = r
        id = m[z-1,y-1,x-1]
        count_labeled += id > 0

        ra,dec,wl = w.wcs_pix2world(x,y,z,1)
        noise_at_wl = np.polyval(p,wl)
        val = d[z-1,y-1,x-1]

        if id <= 0:
            threshold = sigma_grow_threshold*noise_at_wl

            m = cf.grow_segment(d, m, x, y, z, threshold, maxlabel+1)

            N_labeled_new = np.sum(m > 0.) 

            maxlabel += 1

            s = "{:3d} {:3d} {:3d} {:3d} {:6.3f} {:7.2f} {:5.1f} {:6d} "
            print( s.format(i, x,y,z,val,float(wl),val/noise_at_wl, N_labeled_new)  )


            new_regs.append([maxlabel -1, x,y,z,val,float(wl),val/noise_at_wl, N_labeled_new])

    new_regs = np.array(new_regs)

    N = len(c)

    print()
    print("{} out of {} manually labeled objects had associated segments in original map. Added others.".format(count_labeled,N))
    
    return m

In [10]:
if True:
    field = "COSMOSC"
    ifu = "021"

    catfile  = "../data/{}_{}.txt".format(field,ifu)
    mapcube  = "../data/map_{}_{}.fits.gz".format(field,ifu)
    datacube = "../data/sf2outcube_{}_{}.fits.gz".format(field,ifu)
    noisemodel = "../data/sf2outcube_{}_{}.detect_noise_model".format(field,ifu)
    sigma_grow_threshold  = 1.7
    newmapcube = "../data/mmap_{}_{}.fits.gz".format(field,ifu)

    
    #load noise model
    p = np.loadtxt(noisemodel)

    # read manual catalog
    with open(catfile, 'r') as f:
        ll = f.readlines()
    c = []
    for l in ll:
        x,y,z = [int(np.round(float(f))) for f in l.split()[:3]]
        c.append([x,y,z])

    # open data and map cubes
    mhdu = fits.open(mapcube)
    dhdu = fits.open(datacube)
    m = mhdu[0].data
    d = dhdu[0].data
    w = WCS(dhdu[0].header)

    N_labeled_original = np.sum(m > 0.)

    # add segments for manually labeled regions.
    #m = apply_manual_catalog(catfile, mapcube, datacube, noisemodel, sigma_grow_threshold  = 1.7)
    m = apply_manual_catalog(c,m,d,w,p, sigma_grow_threshold  = 1.7, maxlabel = 10000)

    # save new map
    mhdu[0].data = m
    mhdu.writeto(newmapcube, overwrite=True)
    print("Wrote {}.".format(newmapcube))

Maxlabel is:  10000
cidx x   y   z   val      wl[A]   val/noise N_labeled
  7  70  55 157  0.473 3804.53   3.7  59163 
  8  47 109 182  0.570 3854.18   4.3  59477 
  9 100  52 195  0.311 3879.99   2.3  62228 
 10 116  93 203  0.566 3895.88   4.1  62703 
 11  99  84 203  0.456 3895.88   3.3  63168 
 12  98  46 210  0.496 3909.78   3.6  64379 
 13 114  48 210  0.456 3909.78   3.3  64676 
 16  75   9 218  0.531 3925.67   3.8  65200 
 17  44  51 231  0.511 3951.48   3.6  66197 
 18 100  43 240  0.516 3969.36   3.6  72543 
 19   9  61 255  0.588 3999.14   4.0  77587 
 20  52 117 254  0.496 3997.16   3.4  78647 
 24 101  44 330  0.512 4148.08   3.3  85761 
 26 118  60 385  0.695 4257.30   4.3  88829 
 27  96  55 388  0.620 4263.26   3.9  89628 
 28  99  42 388  0.539 4263.26   3.4  91458 
 29 101  56 399  0.584 4285.10   3.6  95677 
 32  42  48 423  0.728 4332.76   4.5  95927 
 33  23  54 423  0.595 4332.76   3.7 100324 
 41 116  88 543  0.576 4571.07   3.5 101912 
 45  41  60 566  0.616 461